In [1]:
from src.vitaldb.fetchingstrategy.Sdk import Sdk
from src.vitaldb.casegenerator import VitalFileOptions
from src.vitaldb.casesplit import split_generator
from src.metrics.standardeviation import StandardDeviation, AbsoluteError
from src.models.baseline import baseline_model
import src.preprocessing.transforms as transforms
import src.preprocessing.filters as filters

import tensorflow as tf

from tensorflow import keras
from keras import layers
from keras.metrics import MeanAbsoluteError
import pandas as pd

In [6]:
frequency = 500
samples = range(1, 100)
train_split = 0.7
validate_split = 0.15
validate_test = 0.15
batching = False

options = VitalFileOptions(
    ['SNUADC/ART'],
    1/frequency
)

train_generator, val_generator, test_generator = split_generator(options, Sdk(),  [0.7, 0.15, 0.15])

dataset_train = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 2), dtype=tf.float64)
    )
)

dataset_val = tf.data.Dataset.from_generator(
    lambda: val_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 2), dtype=tf.float64)
    )
)

2022-06-28 15:52:52.508541: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def preprocess_dataset(dataset: tf.data.Dataset):
    dataset = dataset.filter(filters.has_data)
    dataset = dataset.map(transforms.extract_abp_track)
    dataset = dataset.map(transforms.remove_nan)
    dataset = dataset.map(lambda x: abp_low_pass_graph_adapter(x, frequency))
    dataset = dataset.map(lambda x: extract_clean_windows_graph_adapter(x, frequency, 8, 2))
    dataset= dataset.flat_map(transforms.to_tensor)
    dataset = dataset.filter(lambda x: filters.pressure_out_of_bounds(x, 30, 230))
    dataset = dataset.map(transforms.extract_sbp_dbp_from_abp_window)
    dataset = dataset.map(transforms.scale_array)

    if batching:
        dataset = dataset.map(lambda d, l: (tf.reshape(d, shape=(4000, 1)), l))
        dataset = dataset.batch(20).prefetch(2)
    else:
        dataset = dataset.map(lambda d, l: (tf.reshape(d, shape=(1, 4000)), l))

    return dataset

dataset_train = preprocess_dataset(dataset_train).take(70)
dataset_val = preprocess_dataset(dataset_val).take(15)

In [ ]:
model = baseline_model()
model.summary()
model.compile(optimizer='Adam', loss=keras.losses.MeanAbsoluteError(),
              metrics=[
                  MeanAbsoluteError(),
                  StandardDeviation(AbsoluteError())
              ]
              )